<a href="https://colab.research.google.com/github/Ry02024/SIGNATE/blob/main/%E6%B1%8E%E5%8C%96/%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%82%B9%E3%82%A2%E3%83%97%E3%83%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#streamlit：汎用化

##環境構築

In [1]:
!pip install streamlit -q
!pip install pyngrok -q

In [2]:
import streamlit as st
from pyngrok import ngrok

In [3]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [4]:
from pyngrok import ngrok
import os

# ngrokトークンを設定
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

##実行ファイル

In [5]:
!pip install japanize-matplotlib -q

###util.py

In [9]:
%%writefile util.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import google.generativeai as genai
import chardet

def detect_delimiter(file):
    """
    ファイルのデリミタを自動検出する。
    """
    result = chardet.detect(file.read())
    file.seek(0)  # ファイルポインタをリセット
    sample = file.read(2048).decode(result['encoding'])
    file.seek(0)  # ファイルポインタをリセット
    delimiters = [',', '\t']
    for delimiter in delimiters:
        if delimiter in sample:
            return delimiter
    return ','  # デフォルトはカンマ区切り

def load_data(file, drop_id=True):
    """
    ファイルをアップロードし、デリミタを自動検出してデータフレームを読み込む。
    """
    if file is not None:
        delimiter = detect_delimiter(file)
        data = pd.read_csv(file, delimiter=delimiter)
        if drop_id and 'id' in data.columns:
            data.drop(columns='id', inplace=True)
        return data
    return None

def find_target_column(train_data, test_data):
    """
    訓練データとテストデータの差分から目的変数のカラム名を自動判別する。
    """
    train_columns = set(train_data.columns)
    test_columns = set(test_data.columns)
    target_columns = list(train_columns - test_columns)
    if len(target_columns) == 1:
        return target_columns[0]
    else:
        return None

def visualize_data_analysis(data, target_col):
    """
    データ分析の可視化を実行する。
    """
    if data is not None:
        st.write('基本統計量:')
        st.write(data.describe())

        st.write('特徴量のヒストグラム:')
        fig, ax = plt.subplots()
        data.hist(bins=15, figsize=(15, 10), ax=ax)
        st.pyplot(fig)

        st.write('散布図とカーネル密度推定:')
        pair_plot_fig = sns.pairplot(data, hue=target_col, markers=["o", "s", "D"], palette="bright")
        st.pyplot(pair_plot_fig)

        st.write('相関行列:')
        numeric_data = data.select_dtypes(include=['number'])
        fig, ax = plt.subplots()
        sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', ax=ax)
        st.pyplot(fig)

def summarize_visualizations(data):
    """
    データの可視化結果を要約する。
    """
    summary = ""

    # 基本統計量
    summary += "基本統計量:\n"
    summary += data.describe().to_markdown() + "\n\n"

    # ヒストグラム
    summary += "特徴量のヒストグラム:\n"
    for column in data.columns:
        if data[column].dtype in ['int64', 'float64']:
            summary += f"{column}の分布: 平均={data[column].mean()}, 標準偏差={data[column].std()}, 最大値={data[column].max()}, 最小値={data[column].min()}\n"

    # 相関行列
    numeric_data = data.select_dtypes(include=['number'])
    corr_matrix = numeric_data.corr()
    summary += "\n相関行列:\n"
    summary += corr_matrix.to_markdown() + "\n"

    # 散布図
    summary += "\n散布図:\n"
    summary += "各特徴量間の散布図は以下のように分布しています。\n"

    return summary

def generate_insights(data):
    if data is not None:
        insights = []
        desc = data.describe()

        # 基本統計量からの知見
        insights.append("基本統計量から、各特徴量の平均値、中央値、標準偏差がわかります。これにより、特徴量の分布と中心傾向が理解できます。")

        # 相関行列からの知見
        numeric_data = data.select_dtypes(include=['number'])
        corr_matrix = numeric_data.corr()
        high_corr = corr_matrix[(corr_matrix > 0.7) & (corr_matrix != 1.0)]
        if not high_corr.empty:
            insights.append("相関行列から、以下の特徴量間で強い相関が見られます:")
            insights.append(high_corr.dropna(how='all').dropna(axis=1, how='all').to_markdown())
        else:
            insights.append("相関行列から、特徴量間に強い相関は見られませんでした。")

        return insights
    return []

def train_and_evaluate_model(train_data, test_data, target_col, model_name='RandomForest'):
    """
    モデルを訓練し、評価指標を表示する。
    """
    if train_data is not None and test_data is not None:
        # 特徴量とターゲットに分割
        X_train = train_data.drop(target_col, axis=1)
        y_train = train_data[target_col]
        X_test = test_data

        # 特徴量のスケーリング
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # モデル構築とハイパーパラメータのチューニング
        if model_name == 'RandomForest':
            model = RandomForestClassifier(random_state=42)
            param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30]}
        elif model_name == 'LogisticRegression':
            model = LogisticRegression(random_state=42, max_iter=1000)
            param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

        grid_search = GridSearchCV(model, param_grid, cv=5)
        grid_search.fit(X_train_scaled, y_train)

        # 最適なモデルを取得
        best_model = grid_search.best_estimator_

        # 訓練データに対する予測
        y_train_pred = best_model.predict(X_train_scaled)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_confusion = confusion_matrix(y_train, y_train_pred)

        # 適合率、再現率、F1スコアを計算
        precision = precision_score(y_train, y_train_pred, average='macro')
        recall = recall_score(y_train, y_train_pred, average='macro')
        f1 = f1_score(y_train, y_train_pred, average='macro')

        st.write("訓練データの精度: ", train_accuracy)
        st.write("適合率:", precision)
        st.write("再現率:", recall)
        st.write("F1スコア:", f1)
        fig, ax = plt.subplots()
        sns.heatmap(train_confusion, annot=True, fmt='d', cmap='Blues', ax=ax)
        plt.title("訓練データの混同行列")
        st.pyplot(fig)

        # テストデータに対する予測
        y_test_pred = best_model.predict(X_test_scaled)
        st.write("テストデータの予測結果:")
        st.write(pd.DataFrame({'Prediction': y_test_pred}))

        # 予測結果の要約を生成
        prediction_summary = f"""
        訓練データの精度: {train_accuracy}
        適合率: {precision}
        再現率: {recall}
        F1スコア: {f1}
        """
        return prediction_summary, y_test_pred
    return None, None

def get_visualization_insights(visualization_summary):
    """
    データ分析の可視化結果に基づいた知見を取得する。
    """
    model = genai.GenerativeModel('gemini-pro')
    combined_text = f"""
    以下のデータ分析結果に基づいて、データの詳細な分析を提供してください。

    **データの可視化結果の要約:**
    {visualization_summary}
    """
    response = model.generate_content(combined_text)
    return response.text

def get_insights_from_gemini(summary, y_test_pred):
    """
    Gemini APIを使用して、予測結果に基づいた知見を取得する。
    """
    model = genai.GenerativeModel('gemini-pro')
    combined_text = f"""
    以下の予測結果に基づいて、モデルのパフォーマンスと結果についての詳細な分析を提供してください。

    **予測結果の要約:**
    {summary}

    **テストデータの予測結果:**
    {y_test_pred}
    """
    response = model.generate_content(combined_text)
    return response.text

Overwriting util.py


###streamlit_app.py

In [10]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from util import load_data, visualize_data_analysis, train_and_evaluate_model, get_insights_from_gemini, generate_insights, summarize_visualizations, get_visualization_insights, find_target_column

# Streamlitの設定
st.set_page_config(page_title="データ分析と機械学習アプリ", layout="wide")

st.title('データ分析と機械学習アプリ')
st.write('このアプリはアップロードされたデータセットを分析し、予測モデルを構築します。')

# APIキー入力部分
api_key = st.text_input("APIキーを入力してください:", value="", type="password")

if api_key:
    import google.generativeai as genai
    genai.configure(api_key=api_key)

# ファイルアップローダー
train_file = st.file_uploader("訓練データをアップロードしてください（csvまたはtsv形式）", type=['csv', 'tsv'])
test_file = st.file_uploader("テストデータをアップロードしてください（csvまたはtsv形式）", type=['csv', 'tsv'])

# モデル選択
model_name = st.selectbox("使用するモデルを選択してください:", ['RandomForest', 'LogisticRegression'])

# セッションステートを初期化
if 'visualization_insights' not in st.session_state:
    st.session_state.visualization_insights = None

# チャットボットの初期化
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

# レイアウト設定
col1, col2 = st.columns([3, 1])

with col1:
    if train_file is not None and test_file is not None:
        train_data = load_data(train_file)
        test_data = load_data(test_file, drop_id=True)

        # ターゲット変数の自動判別
        target_col = find_target_column(train_data, test_data)
        if target_col is None:
            st.error("訓練データとテストデータの差分から目的変数を特定できませんでした。")
        else:
            st.write(f"自動検出された目的変数: {target_col}")

            # データの表示
            st.write("訓練データの最初の5行:")
            st.write(train_data.head())
            st.write("テストデータの最初の5行:")
            st.write(test_data.head())

            # 基本統計と分布の表示
            if st.checkbox('可視化によるデータ分析'):
                visualize_data_analysis(train_data, target_col)

                if st.button('Geminiに可視化結果を分析させる'):
                    visualization_summary = summarize_visualizations(train_data)
                    st.session_state.visualization_insights = get_visualization_insights(visualization_summary)

            # セッションステートに保存されたGeminiからの知見を表示
            if st.session_state.visualization_insights:
                st.write("Geminiからの知見:")
                st.write(st.session_state.visualization_insights)

            # モデルの訓練と評価
            if st.button('モデルを訓練して評価'):
                summary, y_test_pred = train_and_evaluate_model(train_data, test_data, target_col, model_name)
                if summary:
                    st.write("モデルのパフォーマンス要約:")
                    st.write(summary)

                    # Geminiを使って知見を取得
                    insights = get_insights_from_gemini(summary, y_test_pred)
                    st.write("Geminiからの知見:")
                    st.write(insights)

with col2:
    st.sidebar.title("Geminiチャットボット")
    query = st.sidebar.text_area("質問を入力してください:", key="query")

    if st.sidebar.button("送信"):
        if api_key:
            model = genai.GenerativeModel('gemini-pro')
            response = model.generate_content(query)
            st.session_state.chat_history.append({"query": query, "response": response.text})
        else:
            st.sidebar.error("APIキーを入力してください。")

    # チャット履歴を表示
    if st.session_state.chat_history:
        for chat in st.session_state.chat_history:
            st.sidebar.write(f"**質問**: {chat['query']}")
            st.sidebar.write(f"**回答**: {chat['response']}")

Overwriting streamlit_app.py


##アプリの起動

In [11]:
# ngrokを介してStreamlitを公開
public_url = ngrok.connect(addr='8501')
print('Public URL:', public_url)
# Streamlitアプリケーションの起動
!streamlit run streamlit_app.py >/dev/null

Public URL: NgrokTunnel: "https://55df-34-23-5-72.ngrok-free.app" -> "http://localhost:8501"
